<a href="https://colab.research.google.com/github/Pavlos01232/Match_Outcome_Prediction/blob/main/Game_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch as t
import torch.nn as n
import pandas as pd
import os
import math


t.set_printoptions(precision=10)
#print("beep boop")
#print("Aston Villa loses")
print("=============================================================")

#GITHUB LOCATION:
#https://github.com/Pavlos01232/Match_Outcome_Prediction
#
#
#
#
#
#
#




#pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0304.csv?raw=true')


#"DEEP learning" just means "hidden" layers


#df.to_csv(r'C:\Users\Pavlos\Desktop\export_dataframe.csv', sep='\t', encoding='utf-8')
#print (df[2])
#file_list = os.listdir('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/')
#df = pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0405.csv?raw=true',sep='\t', lineterminator='\r')
#print(df)

#read function

first = "https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL"
last = ".csv?raw=true"




'''
Starting with an array of data frames,
and an array of column names, make a
single array with the chosen columns
'''




def combine(dataFrames,columnNames):
 t=[]
 for i in range(0,len(dataFrames)):
    theseColumns=dataFrames[i].columns.values[0].split(",")
    for j in range(0 ,len(dataFrames[i])):
      row=dataFrames[i].values[j][0].split(",")
      newEntry=[]
      for k in range(0, len(columnNames)):
         for m in range(0, len(theseColumns)):
             if(columnNames[k]==theseColumns[m] and m<=len(row)):
                newEntry.append(row[m])
      t.append(newEntry)
 return t




'''
reads years 3 to 23,
since the first two files in the training data are formatted incorrectly
converts csv to dataframe
'''

df=[]

for i in range(5, 7):
  result = first + str('{:02.0f}'.format(i)) + str('{:02.0f}'.format(i+1)) + last
  x = pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r')
  df.append(x)




'''
Get an array with each (home) team listed once
from an array of data frames with column "HomeTeam"
'''





def getTeams(df):
 teams=[]
 homeTeams=combine(df,["HomeTeam"])
 for i in range(len(homeTeams)):
  if(len(homeTeams[i])>0):
   found=False
   for j in range(len(teams)):
    if homeTeams[i][0] == teams[j]:
      found=True
      break
   if found:
    continue
   teams.append(homeTeams[i][0])
 return teams


'''
Get the list of teams from the array of data frames called df
and print it to the console
'''


teams=getTeams(df)
print("teams:")
print(teams)





'''
Create an array called Data with just the team names and scores
from the data framees in the array of frames df, and print it

The list ["HomeTeam","AwayTeam","FTHG","FTAG"] can be
made longer if other columnts may be useful to use
'''



Data=combine(df, ["HomeTeam","AwayTeam","FTHG","FTAG"])
print("data: (team names and respective goals scored)")
print(Data)




'''
Get the numerical index of a team name x in the array teams
otherwise just return x
'''

def getIndex(x,teams):
  for i in range(len(teams)):
   if(teams[i]==x):
    return i
  return x


print("Index assigned to Everton:")
print(getIndex("Everton",teams))


'''
Replace any occurrence of names from the array teams
which occur anywhere in A by their actual  numbers
'''

def teamsToNumbers(A,teams):
  B=[]
  for i in range(len(A)):
    B.append([])
    for j in range(len(A[i])):
      B[i].append(getIndex(A[i][j],teams))
  return B


'''
Create Data2 which is a copy of Data but with team names
replaced by their index
'''

print("data2, using the team's index number instead of name")
Data2=teamsToNumbers(Data,teams)
print(Data2)


'''


Assume Data2 has team indices in column 0 and 1 and scores
in cols 2 and 3

A[i] is array of average goals landed per game by tean i
B[i] is array of average goals conceded per game by team i
C is total games played by all teams (twide the number of games)
games[i]=total games played by team i
a[i]*b[j]=first approx of expected goals landed by i when playing
    against j
'''

A=[0]*len(teams)
B=[0]*len(teams)
games=[0]*len(teams)
a=[0]*len(teams)
b=[0]*len(teams)
C=0
totalGames=0

for i in range(len(Data2)):
  if(len(Data2[i])==0):
    continue
  games[Data2[i][0]]+=1
  games[Data2[i][1]]+=1
  A[Data2[i][0]]+=int(Data2[i][2])
  B[Data2[i][0]]+=int(Data2[i][3])
  A[Data2[i][1]]+=int(Data2[i][3])
  B[Data2[i][1]]+=int(Data2[i][2])
  C+=int(Data2[i][0])+int(Data2[i][1])
  totalGames+=2


'''
Initial estimates of a,b,c
'''

for i in range(len(A)):
  a[i]=A[i]*C**(-1/2)

for i in range(len(B)):
   b[i]=B[i]*C**(-1/2)

c=[0.0]





'''
AI training function

The training is by gradient descent, the loss
function will be cross entropy loss function against Poisson
using loss.backward()

The hidden weights at the moment are the entries of a,b,c
the array c has one entry as it is shared for all teams.
These enter into the calculation of mu (which we
call muHome or muAway during training) and are
hidden as they have no direct meaning.

Thus mu as a function of the entries of a,b,c is
learned by training, the weights are the entries
of the three arrays.

For future modification one can make the formula for mu
include additive or multiplicative terms depending
on further entries of the rows of Data2 if one
wants to make the list of useful columns
["HomeTeam","AwayTeam","FTHG","FTAG"]
longer

This will involve including more weight
arrays besides just a,b,c  and they need to
be converted to tensors using requires_grad=True
just as we did for a,b,c
be MUCH smaller than eta since it is shared.

tau is the training rate for c which should



'''



def train(eta,tau):
  loss=t.tensor(0.0)

  '''
tensor versions of weights

this list can be made larger
'''


  atensor=t.tensor(a,requires_grad=True)
  btensor=t.tensor(b,requires_grad=True)
  ctensor=t.tensor(c,requires_grad=True)


  for i in range(len(Data2)):
    if(len(Data2[i])==0):
      continue
    homeTeam=int(Data2[i][0])
    awayTeam=int(Data2[i][1])
    homeGoals=int(Data2[i][2])
    awayGoals=int(Data2[i][3])
    muHome=atensor[homeTeam]*btensor[awayTeam]+ctensor[0]*(atensor[homeTeam]+btensor[awayTeam])
    muAway=btensor[homeTeam]*atensor[awayTeam]+ctensor[0]*(btensor[homeTeam]+atensor[awayTeam])
    loss-=t.log(t.exp(-muHome)*t.pow(muHome,homeGoals)/math.factorial(homeGoals))
    loss-=t.log(t.exp(-muAway)*t.pow(muAway,awayGoals)/math.factorial(awayGoals))
  loss.backward()
  c[0]=c[0]-tau*ctensor.grad
  for i in range(len(a)):
    a[i]=a[i]-eta*atensor.grad[i]
  for i in range(len(btensor)):
    b[i]=b[i]-eta*btensor.grad[i]
  print("Cross-entropy loss vs Poisson"+str(loss))
  print("a:  "+str(a))
  print("b:  "+str(b))
  print("c:  "+str(c))
  print("\n\n\n")
  print("Partial derivatives of loss w/r to hidden weights:")
  print("atensor.grad: "+str(atensor.grad))
  print("btensor.grad: "+str(btensor.grad))
  print("ctensor.grad: "+str(ctensor.grad))
  print("\n\n\n")




'''
Use weights to construct predicted expected goals scored by i
against j and then find probability of k goals scored using
Poisson
'''

def goalProb(i,j,k):
  mu=a[i]*b[j]+c[0]*(a[i]+b[j])
  return math.exp(-mu)*mu**k/math.factorial(k)

'''
Test training a bit
'''

print("est probability before training of 2 landing 2 goals against 3: "+str(goalProb(2,3,2)))
train(0.001,0.00001)
train(0.001,0.00001)
train(0.001,0.00001)
train(0.001,0.00001)
train(0.001,0.00001)
train(0.001,0.00001)
print("probability after training of 2 landing 2 goals against 3: "+str(goalProb(2,3,2)))

print("=============================================================")

i=5

stats=t.tensor(i)
#stats[0]=5
#stats[1]=2
#stats[2]=8
#stats[3]=4
#stats[4]=8
#stats[5]=3

#print(stats)



#t.cuda.memory_allocated(0)/1024**3

teams:
['Aston Villa', 'Everton', 'Fulham', 'Man City', 'Middlesbrough', 'Portsmouth', 'Sunderland', 'West Ham', 'Arsenal', 'Wigan', 'Birmingham', 'Blackburn', 'Charlton', 'Liverpool', 'Man United', 'Newcastle', 'Tottenham', 'West Brom', 'Bolton', 'Chelsea', 'Reading', 'Sheffield United', 'Watford']
data: (team names and respective goals scored)
[['Aston Villa', 'Bolton', '2', '2'], ['Everton', 'Man United', '0', '2'], ['Fulham', 'Birmingham', '0', '0'], ['Man City', 'West Brom', '0', '0'], ['Middlesbrough', 'Liverpool', '0', '0'], ['Portsmouth', 'Tottenham', '0', '2'], ['Sunderland', 'Charlton', '1', '3'], ['West Ham', 'Blackburn', '3', '1'], ['Arsenal', 'Newcastle', '2', '0'], ['Wigan', 'Chelsea', '0', '1'], ['Birmingham', 'Man City', '1', '2'], ['Blackburn', 'Fulham', '2', '1'], ['Charlton', 'Wigan', '1', '0'], ['Liverpool', 'Sunderland', '1', '0'], ['Man United', 'Aston Villa', '1', '0'], ['Newcastle', 'West Ham', '0', '0'], ['Tottenham', 'Middlesbrough', '2', '0'], ['West Brom', '